In [1]:
# Auto Reload Modules
%load_ext autoreload

In [2]:
from utils.data_loader import *

In [3]:
if __name__ == "__main__":
    train_path = r"E:\Kuliah\Kuliah\Kuliah\PRODI\Semester 7\ProSkripCode\data\raw\trainset.csv"
    item_info_path = r"E:\Kuliah\Kuliah\Kuliah\PRODI\Semester 7\ProSkripCode\data\raw\item_info.csv"

    log_cfg = LogConfig(path=train_path, max_click_history=50, slate_size=9)
    item_cfg = ItemConfig(path=item_info_path, item_vec_dim=None)

    df_log  = load_log_table(log_cfg)
    df_item = load_item_table(item_cfg)

    print(df_log.head())
    print(df_item.head())
    print("click_history_ids example:", df_log["click_history_ids"].iloc[0])
    print("exposed_item_ids example:", df_log["exposed_item_ids"].iloc[0])
    print("label_list example:", df_log["label_list"].iloc[0])
    print("item_vec_array shape:", df_item["item_vec_array"].iloc[0].shape)


   user_id                                 user_click_history  \
0        1  30:1580603130,34:1581178937,15:1581178939,28:1...   
1        2  20:1580644762,13:1580644765,1:1580644770,127:1...   
2        3  39:1580772975,6:1580772981,1:1581178309,2:1581...   
3        4  6:1580657608,9:1580657612,15:1580657615,127:15...   
4        5  37:1580743545,36:1580743554,25:1580743556,37:1...   

                                       user_protrait  \
0  64054,21804,80911,36504,8867,7615,54240,37294,...   
1  64054,26565,93755,88510,6344,7615,54240,21927,...   
2  64054,64086,63021,88510,93500,7615,54240,21927...   
3  64054,21531,6599,16721,37078,7615,54240,65505,...   
4  64054,66036,6599,88510,76066,20543,83978,37294...   

                   exposed_items             labels        time  \
0  1,20,28,99,86,119,213,237,164  1,1,1,1,1,1,1,1,1  1582992009   
1  1,4,26,112,86,117,191,234,172  1,1,1,1,1,1,1,1,0  1582992010   
2  22,4,28,48,105,42,193,236,159  1,1,1,0,1,0,0,0,0  1582992014   
3  5

In [ ]:
# train_dqn.py (contoh snippet)
import torch
from torch.optim import Adam

from model.dqn_model import create_q_networks, hard_update, save_model
from model.loss_function import DQNLoss

device = "cuda"

state_dim = 128      # nanti disesuaikan encoder
item_dim = 4         # dari df_item["item_vec_array"]

q, q_target = create_q_networks(state_dim, item_dim, hidden_dim=256, device=device)
optimizer = Adam(q.parameters(), lr=1e-3)
criterion = DQNLoss(gamma=0.99)

for batch in train_loader:  # asumsi sudah ada Dataset
    state      = batch["state"].to(device)      # [B, state_dim]
    item_vec   = batch["item_vec"].to(device)   # [B, item_dim]
    reward     = batch["reward"].to(device)     # [B]
    done       = batch["done"].float().to(device)  # [B]
    next_state = batch["next_state"].to(device) # [B, state_dim]
    next_item_candidate_vecs = batch["next_item_vecs"].to(device) # [B, K, item_dim]
    next_mask  = batch["next_mask"].to(device)  # [B, K] (valid actions)

    # Q(s,a)
    q_sa = q(state, item_vec)                   # [B]

    # Q_target(s', a') untuk semua kandidat, lalu ambil max
    B, K, D = next_item_candidate_vecs.shape
    next_state_rep = next_state.unsqueeze(1).expand(-1, K, -1)   # [B,K,state_dim]
    q_next_all = q_target(
        next_state_rep.reshape(B * K, -1),
        next_item_candidate_vecs.reshape(B * K, -1)
    ).reshape(B, K)

    # masking actions tidak valid (set ke -inf)
    q_next_all = q_next_all.masked_fill(~next_mask, float("-inf"))
    q_next_max, _ = q_next_all.max(dim=1)       # [B]

    loss = criterion(q_sa, reward, done, q_next_max)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # update target network (misal hard update periodik)
    # hard_update(q_target, q)